# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>

#### <center> **Final Project: Batch Processing** </center>
---

**Date**: October, 2025

**Student Name**: Luis Roberto Chávez Mancilla

**Professor**: Pablo Camarillo Ramirez

# Introduction
En la actulidad el crecimiento sin medida del contenido audiovisual ha hecho cada vez mas dificil para las plataformas de streaming y las audiencias comprender las tendecias y las preferencias del publico. 
Ante esto el analis de los datos se vuelve una herramienta fundamental para que la industria del entretenimiento pueda tomar decisiones sobre produccion, localizacion y estrategias de recomendaciones a los usuarios.

En este proyecto tenemos que desarrollar nuestro data pipeline que analice informacion global de programas de television con el objetivo de identificar tendencias relacionadas con la popularidad, calificaciones, el idioma, genero y el pais de origen. El data set utilizado es *10k Popular TV Shows Dataset 1944-2025 (TMDB)* contiene 10 mil registros con metadatos detallados como el titulo, idioma original, paises de origen, generos, metricas de popularidad, promedios de votos y fechas de estreno.

En este proyecto busco mediante el...

# Dataset
Obtuve este dataset de Kaggle:
- https://www.kaggle.com/datasets/riteshswami08/10000-popular-tv-shows-dataset-tmdb

Para este proyecto el modelo seleccionado para nuestro pipeline es un *modelo relacional* usando un container de PostgreSQL como nuestro sistema de gestión.

Creo que en este caso un modelo relacional es lo mas adecuado debido a las relaciones naturales entre las entidades de nuestro data set (series, generos y paises), lo cual facilita la integridad referencial, normalizacion y nos permite podeer realizar analisis complejos mediante operaciones como joins y aggregations.

El esquema propuesto estaría compuesto por 5 tablas principales:
1. Shows (tabla con a información principal)

`id` - int

`name` - str

`original_name` - str

`original_language` - str

`first_air_date` - date(str)

`overview` - str

`popularity` - float

`vote_average` - float

`vote_count` - int

`adult` - int

`poster_path` - str

`backdrop_path` - str

2. Genres (lista de géneros disponibles en el data set)

`genre_id` - int

`genre_name` - str

3. Show genres (tabla de relacion entre shows y generos)

`show_id` - int (fk)

`genre_id` - int (fk)

4. Countries (catálogo de países)

`country_code` - str

`country_name` - str

5. Show countries (relación entre shows y país de origen)

`show_id` - int

`country_code` - str

**Notas:** `genre_ids` y `origin_country` en el CSV vienen como strings tipo lista (ej. `"[18, 35]"` o `"['US']"`). En la etapa de ingestión se parsearán (con `ast.literal_eval` / `json.loads`) y se poblarán las tablas `genres` / `show_genres` y `countries` / `show_countries`. También se crearán columnas derivadas en Transformations (p. ej. `release_year`, `genre_count`, `weighted_popularity`).



# Transformations and Actions
### Create Spark Session

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Project batch processing - Part 1.") \
    .master("spark://spark-master:7077") \
    .config("spark.jars", "/opt/spark/work-dir/jars/postgresql-42.7.8.jar") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()


sc = spark.sparkContext
sc.setLogLevel("INFO")

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

25/10/24 02:12:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Define schema

In [2]:
from robertoman.spark_utils import SparkUtils

# Definición de columnas para la tabla shows (orden exacto del CSV)
shows_schema_columns = [
    ("adult", "boolean"),           # True/False en CSV
    ("backdrop_path", "string"),
    ("genre_ids", "string"),        # parsear después con ast.literal_eval
    ("id", "int"),                 # numeric id
    ("origin_country", "string"),   # parsear después con ast.literal_eval
    ("original_language", "string"),
    ("original_name", "string"),
    ("overview", "string"),
    ("popularity", "float"),
    ("poster_path", "string"),
    ("first_air_date", "string"),   # convertir a date en Transformations
    ("name", "string"),
    ("vote_average", "float"),
    ("vote_count", "int")
]

# Generamos el esquema de PySpark usando SparkUtils
shows_schema = SparkUtils.generate_schema(shows_schema_columns)
shows_schema

StructType([StructField('adult', BooleanType(), True), StructField('backdrop_path', StringType(), True), StructField('genre_ids', StringType(), True), StructField('id', IntegerType(), True), StructField('origin_country', StringType(), True), StructField('original_language', StringType(), True), StructField('original_name', StringType(), True), StructField('overview', StringType(), True), StructField('popularity', FloatType(), True), StructField('poster_path', StringType(), True), StructField('first_air_date', StringType(), True), StructField('name', StringType(), True), StructField('vote_average', FloatType(), True), StructField('vote_count', IntegerType(), True)])

In [3]:
!pwd

/opt/spark/work-dir


### Load CSV

In [3]:
base_path = "/opt/spark/work-dir/data/"

df_shows = spark.read \
    .option("header", "true") \
    .schema(shows_schema) \
    .csv(base_path + "shows/")  

# df_shows.show(5, truncate=False)
df_shows.printSchema()

root
 |-- adult: boolean (nullable = true)
 |-- backdrop_path: string (nullable = true)
 |-- genre_ids: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- origin_country: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_name: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: float (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- first_air_date: string (nullable = true)
 |-- name: string (nullable = true)
 |-- vote_average: float (nullable = true)
 |-- vote_count: integer (nullable = true)



In [6]:
from pyspark.sql.functions import col

total_records = df_shows.count()
print(f"\nTotal de registros: {total_records}\n")

# Contar NULLs por columna usando PySpark
print(f"{'Columna':<25} {'NULLs':<10} {'Porcentaje':<10}")
print("-" * 50)

for column_name in df_shows.columns:
    null_count = df_shows.filter(col(column_name).isNull()).count()
    if null_count > 0:
        percentage = (null_count / total_records * 100)
        print(f"{column_name:<25} {null_count:<10} {percentage:>6.2f}%")



Total de registros: 10989

Columna                   NULLs      Porcentaje
--------------------------------------------------
adult                     989          9.00%
adult                     989          9.00%
backdrop_path             733          6.67%
backdrop_path             733          6.67%
genre_ids                 105          0.96%
genre_ids                 105          0.96%
id                        989          9.00%
id                        989          9.00%
origin_country            190          1.73%
origin_country            190          1.73%
original_language         213          1.94%
original_language         213          1.94%
original_name             234          2.13%
original_name             234          2.13%
overview                  1295        11.78%
overview                  1295        11.78%
popularity                2055        18.70%
popularity                2055        18.70%
poster_path               1510        13.74%
poster_path       

25/10/24 13:25:43 ERROR TaskSchedulerImpl: Lost executor 0 on 172.18.0.4: worker lost: Not receiving heartbeat for 60 seconds


In [ ]:
# Definir columnas críticas (no pueden ser NULL)
critical_columns = ['id', 'name', 'first_air_date', 'genre_ids', 'origin_country']

# Paso 2.1: Eliminar filas donde columnas críticas sean NULL
df_clean = df_shows.dropna(subset=critical_columns)

print(f"\nRegistros eliminados por NULLs en columnas críticas: {df_shows.count() - df_clean.count()}")
print(f"Registros restantes: {df_clean.count()}")

# Paso 2.2: Rellenar NULLs en columnas opcionales con valores por defecto
df_clean = df_clean.fillna({
    'overview': 'No description available',
    'backdrop_path': '',
    'poster_path': '',
    'vote_count': 0,
    'vote_average': 0.0,
    'popularity': 0.0,
    'adult': False,
    'original_language': 'unknown',
    'original_name': ''
})

print("\nNULLs en columnas opcionales rellenados con valores por defecto")
print("=" * 80)

# Paso 2.3: Verificar que no quedan NULLs (100% PySpark)
print("\n✓ Verificando NULLs restantes...")
total_nulls = 0
for column_name in df_clean.columns:
    null_count = df_clean.filter(col(column_name).isNull()).count()
    total_nulls += null_count

print(f"\n✓ Total de NULLs después de limpieza: {total_nulls}")
print(f"✓ Dataset final: {df_clean.count()} registros")
print("\n" + "=" * 80)

Total de registros en el dataset: 10989


Total de registros en el dataset sin NULLs: 7407


# Persistence Data

# DAG

In [12]:
sc.stop()